In [1]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.font_manager as fm
import numpy as np; np.random.seed(42)
import seaborn as sns

import pyLDAvis
import pyLDAvis.gensim
from collections import OrderedDict
from itertools import combinations
import MeCab
from konlpy.tag import *
import nltk
from nltk.corpus import stopwords
from gensim import corpora, models
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.decomposition import NMF, LatentDirichletAllocation, TruncatedSVD
from sklearn.preprocessing import normalize;
from sklearn import decomposition;

/Users/gimjaewon/opt/anaconda3/envs/venv/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
/Users/gimjaewon/opt/anaconda3/envs/venv/lib/python3.7/site-packages/botocore/vendored/requests/packages/urllib3/_collections.py:1: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working
  from collections import Mapping, MutableMapping
/Users/gimjaewon/opt/anaconda3/envs/venv/lib/python3.7/site-packages/scipy/sparse/sparsetools.py:21: DeprecationWarning: `scipy.sparse.sparsetools` is deprecated!
scipy.sparse.sparsetools is a private module for scipy.sparse, and should not be used.
  _deprecated()
/Users/gimjaewon/opt/anaconda3/envs/venv/l

In [2]:
# 데이터 읽어오기 - 특정 기간에 한해서

df = pd.read_csv('adoor_data/sns_feed.csv', encoding='UTF8')
df['created_at'] = pd.to_datetime(df['created_at'])

start_date = pd.Timestamp(2019, 1, 28, 0)
end_date = pd.Timestamp(2019, 4, 1, 0)

mask = (df['created_at'] > start_date) & (df['created_at'] <= end_date)
df = df.loc[mask]

df.head()

,adoor,SNS,SNS 아이디,private,created_at,photo,content
0,2,facebook,김유리,False,2019-02-08,홍보,저와 팀원들이 영혼을 갈아넣은 서비스 베타버전이 드디어 나왔어요!!🤩\n밤잠을 줄여...
1,2,instagram,nnnyu_ri,False,2019-02-08,홍보,https://adoor.app @adoor.team\n저와 팀원들이 영혼을 갈아넣...
2,2,instagram,yulo_mon,True,2019-02-07,"가족, 아기, 강아지",카조꾸또 잇쇼니 토떼모 우레시깟딴데쓰
3,2,instagram,yulo_mon,True,2019-02-25,일상,🙋🏻‍♀️
4,2,instagram,yulo_mon,True,2019-03-02,나들이,필름캐머러 첫 롤 사진들1\n수라미가 준 로모그래피 심플유즈 기본 장착 컬러네거티브...


In [3]:
# 형태소 분석

twitter = Okt()
pos = lambda d: ['/'.join(p) for p in twitter.pos(d, stem=True, norm=True)]

texts_ko = []
content = df.content
docs_ko = ""

for row in content:
    text = row
    if not pd.isna(text):
        docs_ko = docs_ko + text
        for morph in pos(text):
            if (morph.split('/')[1]) in ['Noun'] and len(morph.split('/')[0]) > 1:
                m = []
                m.append(morph)
                texts_ko.append(m)

In [4]:
dictionary_ko = corpora.Dictionary(texts_ko)
dictionary_ko.save('ko.dict')

In [5]:
# tfidf 변환

tf_ko = [dictionary_ko.doc2bow(text) for text in texts_ko]
tfidf_model_ko = models.TfidfModel(tf_ko)
tfidf_ko = tfidf_model_ko[tf_ko]
corpora.MmCorpus.serialize('ko.mm', tfidf_ko)

In [88]:
# LDA 모델 - tfidf 변환을 하지 않은 경우

ntopics, nwords = 5, 7

lda_ko = models.ldamodel.LdaModel(tf_ko, id2word=dictionary_ko, num_topics=ntopics)
lda_ko.print_topics(num_words=nwords)

[(0,
  '0.027*"생각/Noun" + 0.015*"마음/Noun" + 0.010*"무척/Noun" + 0.010*"방학/Noun" + 0.010*"공개/Noun" + 0.010*"거리/Noun" + 0.008*"악당/Noun"'),
 (1,
  '0.017*"그것/Noun" + 0.015*"사용/Noun" + 0.012*"모든/Noun" + 0.010*"오늘/Noun" + 0.008*"아주/Noun" + 0.008*"라이프/Noun" + 0.008*"내일/Noun"'),
 (2,
  '0.018*"진짜/Noun" + 0.013*"일단/Noun" + 0.013*"필름/Noun" + 0.010*"대한/Noun" + 0.010*"일이/Noun" + 0.008*"마음/Noun" + 0.008*"아침/Noun"'),
 (3,
  '0.020*"시간/Noun" + 0.020*"우리/Noun" + 0.015*"아주/Noun" + 0.015*"시스템/Noun" + 0.013*"사진/Noun" + 0.013*"스님/Noun" + 0.010*"방법/Noun"'),
 (4,
  '0.048*"사람/Noun" + 0.012*"학교/Noun" + 0.012*"오늘/Noun" + 0.010*"친구/Noun" + 0.010*"전깃줄/Noun" + 0.010*"그녀/Noun" + 0.010*"그냥/Noun"')]

In [55]:
# LDA 모델 - tfidf 변환을 적용한 경우

lda_ko = models.ldamodel.LdaModel(tfidf_ko, id2word=dictionary_ko, num_topics=ntopics)
lda_ko.print_topics()

[(0,
  '0.026*"사람/Noun" + 0.024*"그것/Noun" + 0.016*"머리/Noun" + 0.011*"가입/Noun" + 0.008*"이수정/Noun" + 0.008*"진짜/Noun" + 0.008*"대한/Noun" + 0.008*"자주/Noun" + 0.008*"이유/Noun" + 0.008*"노래/Noun"'),
 (1,
  '0.020*"친구/Noun" + 0.020*"우리/Noun" + 0.017*"사진/Noun" + 0.015*"일단/Noun" + 0.012*"소리/Noun" + 0.010*"당신/Noun" + 0.010*"모든/Noun" + 0.010*"버전/Noun" + 0.010*"누구/Noun" + 0.010*"거리/Noun"'),
 (2,
  '0.024*"사람/Noun" + 0.019*"시간/Noun" + 0.019*"마음/Noun" + 0.012*"필름/Noun" + 0.012*"사용/Noun" + 0.012*"진짜/Noun" + 0.010*"제일/Noun" + 0.010*"위해/Noun" + 0.010*"공개/Noun" + 0.007*"몹시/Noun"'),
 (3,
  '0.021*"아주/Noun" + 0.017*"스님/Noun" + 0.010*"그녀/Noun" + 0.010*"여자/Noun" + 0.010*"상태/Noun" + 0.010*"어도/Noun" + 0.010*"모두/Noun" + 0.008*"해외/Noun" + 0.008*"시스템/Noun" + 0.007*"바지/Noun"'),
 (4,
  '0.027*"생각/Noun" + 0.017*"오늘/Noun" + 0.012*"학교/Noun" + 0.010*"매일/Noun" + 0.010*"동안/Noun" + 0.010*"베타/Noun" + 0.008*"사람/Noun" + 0.008*"일이/Noun" + 0.008*"기분/Noun" + 0.008*"아침/Noun"')]

In [56]:
ldatopics = lda_ko.show_topics(formatted=False)
ldatopics[0]

(0,
 [('사람/Noun', 0.026419813),
  ('그것/Noun', 0.023703462),
  ('머리/Noun', 0.015955498),
  ('가입/Noun', 0.010818867),
  ('이수정/Noun', 0.008275152),
  ('진짜/Noun', 0.008273189),
  ('대한/Noun', 0.008251676),
  ('자주/Noun', 0.008230993),
  ('이유/Noun', 0.008230561),
  ('노래/Noun', 0.008207712)])

In [89]:
# LSI (Latent Semantic Indexing) 모델

lsi_ko = models.lsimodel.LsiModel(tfidf_ko, id2word=dictionary_ko, num_topics=ntopics)
lsi_ko.print_topics(num_words=nwords)

[(0,
  '-1.000*"사람/Noun" + 0.001*"데스크탑/Noun" + -0.001*"예지/Noun" + -0.001*"통제/Noun" + 0.001*"공포/Noun" + -0.001*"생활/Noun" + -0.001*"벌써/Noun"'),
 (1,
  '-1.000*"생각/Noun" + -0.004*"캐슬/Noun" + -0.004*"음악/Noun" + -0.004*"상공/Noun" + -0.003*"공개/Noun" + -0.003*"최고/Noun" + -0.003*"촉각/Noun"'),
 (2,
  '0.710*"그것/Noun" + 0.654*"오늘/Noun" + -0.220*"아주/Noun" + -0.139*"마음/Noun" + -0.005*"이틀/Noun" + -0.005*"퓨저/Noun" + -0.005*"위로/Noun"'),
 (3,
  '-0.712*"오늘/Noun" + 0.615*"그것/Noun" + -0.264*"아주/Noun" + 0.208*"마음/Noun" + 0.008*"대화/Noun" + -0.006*"고생/Noun" + -0.006*"세상/Noun"'),
 (4,
  '-0.785*"마음/Noun" + -0.568*"아주/Noun" + -0.174*"그것/Noun" + -0.169*"오늘/Noun" + -0.009*"해도/Noun" + 0.008*"처음/Noun" + 0.007*"노래/Noun"')]

In [58]:
# HDP (Hierarchical Dirichlet Process) 모델 - n_topics printed ONLY

hdp_ko = models.hdpmodel.HdpModel(tfidf_ko, id2word=dictionary_ko)
hdp_ko.print_topics(ntopics, nwords)

/Users/gimjaewon/opt/anaconda3/envs/venv/lib/python3.7/site-packages/gensim/models/hdpmodel.py:467: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  start_time = time.clock()


[(0,
  '0.009*자랑/Noun + 0.009*여러/Noun + 0.008*이메일/Noun + 0.007*경험/Noun + 0.007*허리/Noun'),
 (1,
  '0.011*부스/Noun + 0.009*송도/Noun + 0.009*진짜/Noun + 0.008*내일/Noun + 0.008*작년/Noun'),
 (2,
  '0.007*나이트/Noun + 0.006*기한/Noun + 0.006*터득/Noun + 0.006*핬다/Noun + 0.006*채널/Noun'),
 (3,
  '0.008*기억/Noun + 0.008*정자/Noun + 0.007*고민/Noun + 0.007*첫인상/Noun + 0.006*신세계/Noun'),
 (4,
  '0.009*의미/Noun + 0.006*이번/Noun + 0.006*더욱/Noun + 0.006*대한/Noun + 0.006*페이보릿/Noun')]

In [59]:
# Bag of Words

bow = tfidf_model_ko[dictionary_ko.doc2bow(texts_ko[0])]
sorted(lda_ko[bow], key=lambda x: x[1], reverse=True)

[(1, 0.59562576),
 (4, 0.10425308),
 (0, 0.10004156),
 (3, 0.10004048),
 (2, 0.10003912)]

In [60]:
sorted(hdp_ko[bow], key=lambda x: x[1], reverse=True)

[(27, 0.5037132359856408)]

In [61]:
bow = tfidf_model_ko[dictionary_ko.doc2bow(texts_ko[1])]
sorted(lda_ko[bow], key=lambda x: x[1], reverse=True)

[(4, 0.5999499),
 (3, 0.10001322),
 (0, 0.10001288),
 (1, 0.100012705),
 (2, 0.10001133)]

In [62]:
sorted(hdp_ko[bow], key=lambda x: x[1], reverse=True)

[(63, 0.5034787282764315)]

In [63]:
# Coherence 수준 측정 - LDA 모델

def compute_coherence_values(dictionary, corpus, texts, limit, start=2, step=3):
    coherence_values = []
    model_list = []
    for num_topics in range(start, limit, step):
        model = models.LdaModel(corpus=tf_ko, num_topics=num_topics, id2word=dictionary_ko)
        model_list.append(model)
        coherencemodel = models.CoherenceModel(model=model, texts=texts_ko, dictionary=dictionary_ko, coherence='c_v')
        coherence_values.append(coherencemodel.get_coherence())

    return model_list, coherence_values

In [64]:
coherence_model_lda = models.CoherenceModel(model=lda_ko, texts=texts_ko, dictionary=dictionary_ko, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.8499339389684447


In [65]:
# model_list, coherence_values = compute_coherence_values(
#     dictionary=dictionary_ko, 
#     corpus=tf_ko, 
#     texts=texts_ko, 
#     start=1, 
#     limit=5, 
#     step=1)

In [66]:
# limit=10; start=1; step=1;
# x = range(start, limit, step)
# plt.rcParams["font.size"] = 20
# plt.figure(figsize=(9,5))
# plt.plot(x, coherence_values)
# plt.xlabel("Num Topics")
# plt.ylabel("Coherence score")
# plt.legend(("coherence_values"), loc='best')
# plt.show()

In [67]:
# for m, cv in zip(x, coherence_values):
#     print("Num Topics =", m, " has Coherence Value of", round(cv, 4))

In [68]:
# optimal_model = model_list[1]
# model_topics = optimal_model.show_topics(formatted=False)
# optimal_model.print_topics(num_words=10)

In [69]:
# # Topics Sentence Formation

# def format_topics_sentences(ldamodel=lda_ko, corpus=tf_ko, texts=df['content'].values.astype('U')):
   
#     sent_topics_df = pd.DataFrame()

   
#     for i, row in enumerate(ldamodel[corpus]):
#         row = sorted(row, key=lambda x: (x[1]), reverse=True)
#         # Get the Dominant topic, Perc Contribution and Keywords for each document
#         for j, (topic_num, prop_topic) in enumerate(row):
#             if j == 0:  # -- dominant topic
#                 wp = ldamodel.show_topic(topic_num)
#                 topic_keywords = ", ".join([word for word, prop in wp])
#                 sent_topics_df = sent_topics_df.append(pd.Series([int(topic_num), round(prop_topic,4), topic_keywords]), ignore_index=True)
#             else:
#                 break
#     sent_topics_df.columns = ['Dominant_Topic', 'Perc_Contribution', 'Topic_Keywords']

#     # Add original text to the end of the output
    
#     contents = pd.Series(texts)
#     sent_topics_df = pd.concat([sent_topics_df, contents], axis=1)
#     return(sent_topics_df)
    

# df_topic_sents_keywords = format_topics_sentences(ldamodel=optimal_model, corpus=tf_ko, texts=df['content'].values.astype('U'))


# df_dominant_topic = df_topic_sents_keywords.reset_index()
# df_dominant_topic.columns = ['Document_No', 'Dominant_Topic', 'Topic_Perc_Contrib', 'Keywords', 'Text']


# df_dominant_topic.head(5)

In [70]:
path='/Library/Fonts/NanumGothic.ttf'
font_name = fm.FontProperties(fname=path, size=50).get_name()
matplotlib.rc('font', family=font_name)
matplotlib.rc('axes', unicode_minus=False)
plt.rc('font', family=font_name)
plt.rc('axes', unicode_minus=False)

In [71]:
# LDA Model w/ 3 Topics

data_lda = {i: OrderedDict(lda_ko.show_topic(i, 5)) for i in range(ntopics)}
df_lda = pd.DataFrame(data_lda)
df_lda = df_lda.fillna(0).T
print(df_lda.shape)
df_lda

(5, 24)


,사람/Noun,그것/Noun,머리/Noun,가입/Noun,이수정/Noun,친구/Noun,우리/Noun,사진/Noun,일단/Noun,소리/Noun,...,아주/Noun,스님/Noun,그녀/Noun,여자/Noun,상태/Noun,생각/Noun,오늘/Noun,학교/Noun,매일/Noun,동안/Noun
0,0.026420,0.023703,0.015956,0.010819,0.008275,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
1,0.000000,0.000000,0.000000,0.000000,0.000000,0.019604,0.019554,0.017204,0.014801,0.012456,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
2,0.023949,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000
3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.021489,0.016831,0.009852,0.009823,0.009819,0.000000,0.000000,0.00000,0.000000,0.000000
4,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.026657,0.017214,0.01243,0.010065,0.010057


In [72]:
# 3-topic LDA 모델 - MDS Mapping

corpus_lda = lda_ko[tfidf_ko]

pyLDAvis.enable_notebook()
panel = pyLDAvis.gensim.prepare(lda_ko, corpus_lda, dictionary_ko, mds='tsne')
panel

PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      150.222153  -72.269554       1        1  38.344582
1       51.163364 -159.968018       2        1  21.981058
2      -73.144554   21.523815       3        1  19.657656
3      -66.410133 -110.606438       4        1  10.008357
0       54.417324   12.443444       5        1  10.008353, topic_info=        Term       Freq      Total Category  logprob  loglift
117  사람/Noun  13.000000  13.000000  Default  30.0000  30.0000
66   그것/Noun   3.000000   3.000000  Default  29.0000  29.0000
38   생각/Noun  13.000000  13.000000  Default  28.0000  28.0000
11   친구/Noun   5.000000   5.000000  Default  27.0000  27.0000
46   우리/Noun   5.000000   5.000000  Default  26.0000  26.0000
..       ...        ...        ...      ...      ...      ...
419  예지/Noun   0.694598   2.401026   Topic5  -5.1742   1.0614
58   진짜/Noun   1.015140   4.486930   Topic5  -4.7947   0.8156
249  오늘/Noun   0.711651   9.194336   Topic5  -5.1499  -0.2570
134  걸음/Noun   0.384098   1.541497   Topic5  -5.7666   0.9121
118  의미/Noun   0.385340   2.248707   Topic5  -5.7634   0.5378

[217 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
229       5  0.518483   가입/Noun
125       1  0.581786   가장/Noun
125       3  0.290893   가장/Noun
59        3  0.858512  갑자기/Noun
540       2  0.520585   개강/Noun
...     ...       ...       ...
193       4  0.827523   허슬/Noun
39        3  0.570321   헤헤/Noun
708       5  0.793549   혼자/Noun
230       3  0.422753   후회/Noun
230       4  0.422753   후회/Noun

[195 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[5, 2, 3, 4, 1])

In [73]:
def getNVM_lemma(text):
    tokenizer = MeCab.Tagger()
    parsed = tokenizer.parse(text)
    word_tag = [w for w in parsed.split("\n")]
    pos = []
    tags = ['NNG','NNP','VV','VA', 'VX', 'VCP','VCN']
    for word_ in word_tag[:-2]:
        word = word_.split("\t")
        tag = word[1].split(",")
        if(len(word[0]) < 2) or ("게" in word[0]):
            continue
        if(tag[-1] != '*'):
            t = tag[-1].split('/')
            if(len(t[0]) > 1 and ('VV' in t[1] or 'VA' in t[1] or 'VX' in t[1])):
                pos.append(t[0])
        else:
            if(tag[0] in tags):
                pos.append(word[0])
    return pos

In [74]:
vectorizer = CountVectorizer(tokenizer=getNVM_lemma, min_df=2)
x_counts = vectorizer.fit_transform(df['content'].values.astype('U'))
print( "Created %d X %d document-term matrix" % (x_counts.shape[0], x_counts.shape[1]) )
transformer = TfidfTransformer(smooth_idf=False);
x_tfidf = transformer.fit_transform(x_counts);

Created 135 X 137 document-term matrix


In [75]:
terms = vectorizer.get_feature_names()
print("Vocabulary has %d distinct terms" % len(terms))

Vocabulary has 137 distinct terms


In [83]:
xtfidf_norm = normalize(x_tfidf, norm='l1', axis=1)
model = NMF(n_components=5, init='nndsvd');
model.fit(xtfidf_norm)

NMF(init='nndsvd', n_components=5)

In [84]:
def get_nmf_topics(model, n_top_words):
    
    feat_names = vectorizer.get_feature_names()
    
    word_dict = {};
    for i in range(num_topics):
        
        words_ids = model.components_[i].argsort()[:-10 - 1:-1]
        words = [feat_names[key] for key in words_ids]
        word_dict['Topic # ' + '{:02d}'.format(i+1)] = words;
    
    return pd.DataFrame(word_dict);

In [85]:
num_topics = 1
nmf_df = get_nmf_topics(model, 2)
nmf_df

,Topic # 01
0,날씨
1,생일
2,시작
3,기념
4,카메라
5,필름
6,개강
7,기분
8,지나
9,열리


In [90]:
num_topics = 5
nmf_df = get_nmf_topics(model, 5)
nmf_df

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05
0,날씨,도어,민윤기,최고,친구
1,생일,소통,노래,재재,생각
2,시작,관심,라이프,강아지,고맙
3,기념,관계,디자인,낌새,서비스
4,카메라,공유,이렇,자르,언니
5,필름,오늘,돌리,보내,강아지
6,개강,보내,재미,방법,필름
7,기분,그리,생기,머리,새벽
8,지나,공개,사람,보이,통제
9,열리,방학,들리,모르,머러


In [80]:
num_topics = 10
nmf_df = get_nmf_topics(model, 5)
nmf_df

,Topic # 01,Topic # 02,Topic # 03,Topic # 04,Topic # 05,Topic # 06,Topic # 07,Topic # 08,Topic # 09,Topic # 10
0,날씨,도어,민윤기,최고,친구,생각,언니,노래,강아지,서비스
1,생일,소통,이렇,재재,필름,통제,새벽,라이프,이렇,고맙
2,시작,관심,돌리,보내,만들,세상,기억,디자인,재미,필름
3,기념,관계,기다리,자르,사용,필름,이미지,생기,재재,나오
4,카메라,공유,재미,낌새,공유,사진,엄청나,사람,보내,그리
5,필름,오늘,다가오,방법,소통,머러,붙이,돌리,낌새,흘리
6,개강,보내,아니,보이,사람,돌리,교수,들리,자르,머러
7,기분,그리,사진,머리,머러,기대,학교,이틀,허리,공유
8,지나,공개,느끼,모르,버전,학교,기대,존재,머리,만들
9,열리,방학,프로그램,따르,베타,방법,방법,그러,방법,사진
